In [1]:
import pandas as pd
import mlflow

In [2]:
df = pd.read_csv('../Dataset/retail_sales.csv')

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

In [4]:
df.set_index('Date', inplace=True)

In [ ]:
df.info()

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, shuffle=False)


In [ ]:
train

In [ ]:
from matplotlib import pyplot as plt

df.plot(title='Retail Sales', figsize=(12, 8), fontsize=14)


In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['Sales'] = scaler.fit_transform(df[['Sales']])

In [10]:
import numpy as np

def create_sequences(data, index, window_size):
    X, y, timestamps = [], [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
        timestamps.append(index[i + window_size])
    return np.array(X), np.array(y), np.array(timestamps)

window_size=24

train_X, train_y, train_ts = create_sequences(train['Sales'].values, train.index, window_size)
test_X, test_y, test_ts = create_sequences(test['Sales'].values, test.index, window_size)




In [11]:
X_train, X_test = train_X, test_X
y_train, y_test = train_y, test_y
ts_train, ts_test = train_ts, test_ts

In [12]:
import numpy as np

def save_preprocessed_data(filepath, X_train, y_train, ts_train, X_test, y_test, ts_test):
    np.savez(filepath,
             X_train=X_train,
             y_train=y_train,
             ts_train=ts_train,
             X_test=X_test,
             y_test=y_test,
             ts_test=ts_test)


In [13]:
save_preprocessed_data("retail_sales_preprocessed.npz", 
                       X_train, y_train, ts_train, 
                       X_test, y_test, ts_test)


In [14]:
def load_preprocessed_data(filepath):
    data = np.load(filepath, allow_pickle=True)
    return (data['X_train'], data['y_train'], data['ts_train'],
            data['X_test'], data['y_test'], data['ts_test'])


In [15]:
X_train, y_train, ts_train, X_test, y_test, ts_test = load_preprocessed_data("retail_sales_preprocessed.npz")


In [ ]:
print("Train lengths")
print(f"train = {len(train)}")
print(f"X_train = {len(X_train)}")

print("Test length")
print(f"test = {len(test)}")
print(f"X_test = {len(X_test)}")

# MLFLOW

In [ ]:
import mlflow
import mlflow.tensorflow
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from kerastuner.tuners import RandomSearch

# Enable autologging
mlflow.tensorflow.autolog()

# Define model builder for KerasTuner
def build_model(hp):
    model = Sequential()
    
    # Choose model type
    model_type = hp.Choice('model_type', ['LSTM', 'GRU', 'Dense'])
    units = hp.Int('units', min_value=32, max_value=128, step=16)
    layers = hp.Int('num_layers', 1, 3)
    
    for i in range(layers):
        return_seq = i < layers - 1
        if model_type == 'LSTM':
            model.add(LSTM(units, activation='relu', return_sequences=return_seq,
                           input_shape=(window_size, 1) if i == 0 else None))
        elif model_type == 'GRU':
            model.add(GRU(units, activation='relu', return_sequences=return_seq,
                          input_shape=(window_size, 1) if i == 0 else None))
        else:
            # Dense model flattens and feeds to Dense layers
            if i == 0:
                model.add(tf.keras.layers.Flatten(input_shape=(window_size, 1)))
            model.add(Dense(units, activation='relu'))
    
    model.add(Dense(1))  # Output layer
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
        loss='mse'
    )
    
    return model

# Setup KerasTuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='mlruns_tuning',
    project_name='degradation_prediction'
)

# Search for best hyperparams (inside mlflow run)
with mlflow.start_run(run_name="hyperparam_tuning"):
    tuner.search(X_train, y_train,
                 epochs=10,
                 batch_size=32,
                 validation_data=(X_test, y_test))

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Log the best model explicitly
mlflow.tensorflow.log_model(best_model, "best_model")


# Tensorflow

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(50, activation='relu', input_shape=(window_size, 1)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
# Predict
y_pred = model.predict(X_test)

In [ ]:
# Inverse scale (optional)
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

# Plot with datetime index
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))
plt.plot(ts_test, y_test_rescaled, label='Actual')
plt.plot(ts_test, y_pred_rescaled, label='Predicted')
plt.xlabel("Timestamp")
plt.ylabel("Sales")
plt.title("LSTM Forecast vs Actual")
plt.legend()
plt.tight_layout()
plt.show()

In [30]:
output = pd.DataFrame({
    'Actual': y_test_rescaled.flatten(),
    'Predicted': y_pred_rescaled.flatten()
})

In [ ]:
output

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))
plt.plot(y_test_rescaled, label='Actual')
plt.plot(y_pred_rescaled, label='Predicted')
plt.legend()
plt.title("LSTM Forecast vs Actual Sales")
plt.show()
